In [65]:
import re
import sys
import nltk
import string
from math import sqrt, log
from collections import defaultdict
from itertools import chain, product
from nltk import word_tokenize as tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer as stemmer
import numpy as np
from itertools import groupby
from collections import Counter
from nltk.collocations import *
from nltk.stem.porter import PorterStemmer
import pandas as pd
import textfeatures as tf
from sklearn import preprocessing
from sklearn import metrics

In [66]:
def compute_average_word_length(sentence):
    return np.mean([len(words) for words in sentence.split()])

def compute_average_sentence_length(sentence):
    sentence = re.split("(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s", sentence)
    return np.mean([len(words) for words in sentence])

def freq_of_words_great_sent_len(sentence):
    result = []
    avg_word_len = compute_average_word_length(sentence)
    # sentence = re.split("(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s", sentence)
    sentence = Counter(sentence.split())
    for key, value in sentence.items():
        if len(key) > avg_word_len:
            result.append(value)
#             print (key, value)
    return sum(result)

def tokenize(sentence):
    return re.split(r"[^0-9A-Za-z\-'_]+", sentence)

def compute_yules_k_for_text(sentence):
    tokens = tokenize(sentence)
    counter = Counter(token.upper() for token in tokens)

    #compute number of word forms in a given sentence/text
    m1 = sum(counter.values())
    m2 = sum([frequency ** 2 for frequency in counter.values()])

    #compute yules k measure and return the value
    yules_k = 10000/((m1 * m1) / (m2 - m1))
    return yules_k


def words_in_sentence(sentence):
    w = [words.strip("0123456789!:,.?()[]{}") for words in sentence.split()]
    return filter(lambda x: len(x) > 0, w)

def compute_yules_i_for_text(sentence):
    dictionary = {}
    stemmer = PorterStemmer()

    for word in words_in_sentence(sentence):
        word = stemmer.stem(word).lower()
        try:
            dictionary[word] += 1
        except:
            dictionary[word] = 1

    m1 = float(len(dictionary))
    m2 = sum([len(list(grouped_values)) * (frequency ** 2) for frequency, grouped_values in groupby(sorted(dictionary.values()))])

    # compute yules i and return the value
    try:
        yules_i = (m1 * m1) / (m2 - m1)
        return yules_i
    except ZeroDivisionError:
        return 0

In [67]:
raw_data = pd.read_csv("scores.csv")
# raw_data = raw_data.sample(frac=0.01).reset_index(drop=True)
raw_data = raw_data.sample(frac=1).reset_index(drop=True)

def remove_stopwords(text):
    stopwords = nltk.corpus.stopwords.words('english')
    content = [w for w in text if w.lower() not in stopwords]
    return content

In [68]:
raw_data["yules_i_measure_abs"] = np.nan
raw_data["yules_i_measure_abs"] = raw_data.apply(lambda x: compute_yules_i_for_text(x['abstract']), axis=1)

raw_data["avg_word_len_abs"] = np.nan
raw_data["avg_word_len_abs"] = raw_data.apply(lambda x: compute_average_word_length(x['abstract']), axis=1)

raw_data["avg_sen_len_abs"] = np.nan
raw_data["avg_sen_len_abs"] = raw_data.apply(lambda x: compute_average_sentence_length(x['abstract']), axis=1)

raw_data["freq_words_gt_sen_len_abs"] = np.nan
raw_data["freq_words_gt_sen_len_abs"] = raw_data.apply(lambda x: freq_of_words_great_sent_len(x['abstract']), axis=1)

raw_data.head()

,abstract,review,semantic_similarity,abstract_clean,review_clean,semantic_similarity_aftercleaning,yules_i_measure_abs,avg_word_len_abs,avg_sen_len_abs,freq_words_gt_sen_len_abs
0,"We propose a new, multi-component energy funct...",Quick summary:\r\nThis paper proposes an energ...,[[0.86421597]],propose new multi component energy function ...,quick summary paper propose energy base form...,[[0.84397495]],30.941406,5.766423,153.500000,63
1,Supervised learning depends on annotated examp...,This paper focuses on the learning-from-crowds...,[[0.9104235]],supervised learn depend annotate example grin...,paper focus learn crowd problem noisy labe...,[[0.89139616]],30.090020,5.872038,119.916667,104
2,Labeled text classification datasets are typic...,The draft proposes an approach to cross-lingua...,[[0.8746078]],labeled text classification dataset typically ...,draft propose approach cross lingual text cla...,[[0.85585344]],13.585965,5.086207,116.888889,63
3,We propose a novel way of reducing the number ...,This paper examines sparse connection patterns...,999999999,propose novel way reduce numb parameter storag...,paper examine sparse connection pattern upper ...,[[0.79835767]],23.918644,5.618321,172.400000,61
4,Actor-critic methods solve reinforcement learn...,The paper presents a clever trick for updating...,999999999,actor critic method solve reinforcement learn...,paper present clever trick update actor actor ...,[[0.8510821]],13.005803,5.389937,112.111111,71


In [69]:
data = raw_data.copy()
data['similarity_score'] = pd.to_numeric(data['semantic_similarity_aftercleaning'].str.replace('[','').str.replace(']',''))
data = data[data['similarity_score'] != 999999999.0].copy()
data['similarity_score'] = data['similarity_score'] * 100

In [70]:
tf.word_count(data,"abstract","word_cnt")
tf.char_count(data,"abstract","char_len")
tf.avg_word_length(data,"abstract","avg_wrd_length")
tf.stopwords_count(data,"abstract","stopwords_cnt")
tf.numerics_count(data,"abstract","num_len")

,abstract,review,semantic_similarity,abstract_clean,review_clean,semantic_similarity_aftercleaning,yules_i_measure_abs,avg_word_len_abs,avg_sen_len_abs,freq_words_gt_sen_len_abs,similarity_score,word_cnt,char_len,avg_wrd_length,stopwords_cnt,num_len
0,"We propose a new, multi-component energy funct...",Quick summary:\r\nThis paper proposes an energ...,[[0.86421597]],propose new multi component energy function ...,quick summary paper propose energy base form...,[[0.84397495]],30.941406,5.766423,153.500000,63,84.397495,137,926,5.766423,49,0
1,Supervised learning depends on annotated examp...,This paper focuses on the learning-from-crowds...,[[0.9104235]],supervised learn depend annotate example grin...,paper focus learn crowd problem noisy labe...,[[0.89139616]],30.090020,5.872038,119.916667,104,89.139616,212,1450,5.872038,63,0
2,Labeled text classification datasets are typic...,The draft proposes an approach to cross-lingua...,[[0.8746078]],labeled text classification dataset typically ...,draft propose approach cross lingual text cla...,[[0.85585344]],13.585965,5.086207,116.888889,63,85.585344,176,1060,5.086207,69,0
3,We propose a novel way of reducing the number ...,This paper examines sparse connection patterns...,999999999,propose novel way reduce numb parameter storag...,paper examine sparse connection pattern upper ...,[[0.79835767]],23.918644,5.618321,172.400000,61,79.835767,131,866,5.618321,46,0
4,Actor-critic methods solve reinforcement learn...,The paper presents a clever trick for updating...,999999999,actor critic method solve reinforcement learn...,paper present clever trick update actor actor ...,[[0.8510821]],13.005803,5.389937,112.111111,71,85.108210,159,1017,5.389937,61,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2743,Deep learning models require extensive archite...,The paper proposes an extension of the Neural ...,999999999,deep learn model require extensive architectur...,paper propose extension neural architecture se...,[[0.85404795]],16.524831,5.812245,138.166667,112,85.404795,244,1669,5.812245,77,0
2744,"We consider neural network training, in applic...",The paper is well-written which makes it easy ...,[[0.89319736]],consider neural network train application pos...,paper good write easy understand main thrust...,[[0.8911872]],11.806375,5.139013,136.600000,85,89.118720,224,1375,5.139013,93,0
2745,Adversarial feature learning (AFL) is one of t...,- The authors propose the use of multiple adve...,[[0.9064467]],adversarial feature learn afl promise way ex...,author propose use multiple adversary random ...,[[0.8601346]],13.690000,5.808612,177.750000,94,86.013460,210,1429,5.808612,80,0
2746,Regularization is a big issue for training dee...,the paper adapts the information bottleneck me...,[[0.89342827]],regularization big issue train deep neural net...,paper adapt information bottleneck method prob...,[[0.7793288]],28.543210,5.960784,141.000000,48,77.932880,102,709,5.960784,37,0


In [71]:
columns = ['yules_i_measure_abs', 'avg_word_len_abs', 'avg_sen_len_abs', 'freq_words_gt_sen_len_abs', 'similarity_score',
          'word_cnt', 'char_len', 'avg_wrd_length', 'stopwords_cnt', 'num_len']
df = data[columns].copy()
x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
#df = pd.DataFrame(x_scaled)
df=pd.DataFrame(x_scaled, columns=df.columns)

In [72]:
training_set = data.sample(frac=0.8, random_state = 1)
test_set = data.loc[~data.index.isin(training_set.index)]
test_set.head()

training_set = df.sample(frac=0.8, random_state = 1)
test_set = df.loc[~df.index.isin(training_set.index)]
test_set.head()

,yules_i_measure_abs,avg_word_len_abs,avg_sen_len_abs,freq_words_gt_sen_len_abs,similarity_score,word_cnt,char_len,avg_wrd_length,stopwords_cnt,num_len
14,0.265512,0.667844,0.829997,0.362416,0.915270,0.299703,0.365145,0.667844,0.231343,0.0
15,0.159442,0.274419,0.401851,0.409396,0.828280,0.412463,0.416782,0.274419,0.365672,0.0
18,0.134984,0.696577,0.498123,0.395973,0.767135,0.400593,0.484555,0.696577,0.313433,0.0
20,0.328817,0.427072,0.311583,0.234899,0.875312,0.264095,0.287690,0.427072,0.231343,0.0
25,0.161022,0.388069,0.216458,0.563758,0.859142,0.543027,0.566160,0.388069,0.477612,0.0


In [73]:
data_columns = ['yules_i_measure_abs', 'avg_word_len_abs',
       'avg_sen_len_abs', 'freq_words_gt_sen_len_abs']

columns = ['yules_i_measure_abs', 'avg_word_len_abs', 'avg_sen_len_abs', 'freq_words_gt_sen_len_abs',
          'word_cnt', 'char_len', 'avg_wrd_length', 'stopwords_cnt', 'num_len']


# training_data = training_set.as_matrix(columns = data_columns)
training_data = training_set[data_columns].to_numpy()# .as_matrix(columns = data_columns)
nan_locs = np.isnan(training_data)
training_data[nan_locs] = 0

training_target = training_set['similarity_score'].values
nan_locs = np.isnan(training_target)
training_target[nan_locs] = 0

test_data = test_set[data_columns].to_numpy()# .as_matrix(columns = data_columns)
nan_locs = np.isnan(test_data)
test_data[nan_locs] = 0

test_target = test_set['similarity_score'].values
nan_locs = np.isnan(test_target)
test_target[nan_locs] = 0

In [74]:
train_data_array = training_data.copy()
train_class_array = training_target.copy()
test_data_array = test_data.copy()
test_class_array = test_target.copy()

In [75]:
from sklearn.neural_network import MLPRegressor
reg = MLPRegressor()
reg.fit(training_data, training_target)

print('R squared: ' + str(reg.score(test_data, test_target)))
pred = reg.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: 0.09736815376029817
MAE: 0.09681168668436005
MSE: 0.01613762316897446


In [76]:
from sklearn.svm import SVR
svr = SVR()
svr.fit(training_data, training_target)

print('R squared: ' + str(svr.score(test_data, test_target)))
pred = svr.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: 0.12083272647376608
MAE: 0.09569713013126403
MSE: 0.015718113893018346


In [77]:
from sklearn.kernel_ridge import KernelRidge
kr = KernelRidge()
kr.fit(training_data, training_target)

print('R squared: ' + str(kr.score(test_data, test_target)))
pred = kr.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: -0.622802514678054
MAE: 0.13460794179775187
MSE: 0.02901313040154366


In [78]:
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor()
dt.fit(training_data, training_target)

print('R squared: ' + str(dt.score(test_data, test_target)))
pred = dt.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: -0.02427441258220786
MAE: 0.1030722315686996
MSE: 0.018312398970559727


In [79]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 2000)
rf.fit(training_data, training_target)

print('R squared: ' + str(rf.score(test_data, test_target)))
pred = rf.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: 0.09653160340839584
MAE: 0.09557588282393557
MSE: 0.016152579360025232


In [80]:
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor()
gb.fit(training_data, training_target)

print('R squared: ' + str(gb.score(test_data, test_target)))
pred = gb.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: 0.15165164255378272
MAE: 0.09306300551973408
MSE: 0.015167120643392318


In [81]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

pr = Pipeline([('poly', PolynomialFeatures(degree = 3)),
              ('linear', LinearRegression(fit_intercept = False))])

pr.fit(training_data, training_target)

print('R squared: ' + str(pr.score(test_data, test_target)))
pred = pr.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: 0.13146911554279306
MAE: 0.09562408766059806
MSE: 0.015527952157212524


In [82]:
from sklearn.linear_model import Lasso
lm = Lasso()
lm.fit(training_data, training_target)

print('R squared: ' + str(lm.score(test_data, test_target)))
pred = lm.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: -0.008909479954041899
MAE: 0.102671836181547
MSE: 0.018037698389361555


In [83]:
from sklearn.linear_model import ElasticNet
en = ElasticNet()
en.fit(training_data, training_target)

print('R squared: ' + str(en.score(test_data, test_target)))
pred = en.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: -0.008909479954041899
MAE: 0.102671836181547
MSE: 0.018037698389361555


In [84]:
from sklearn.linear_model import LassoLars
ll = LassoLars()
ll.fit(training_data, training_target)

print('R squared: ' + str(ll.score(test_data, test_target)))
pred = ll.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: -0.008909479954041899
MAE: 0.102671836181547
MSE: 0.018037698389361555


In [85]:
from sklearn.linear_model import BayesianRidge
br = BayesianRidge()
br.fit(training_data, training_target)

print('R squared: ' + str(br.score(test_data, test_target)))
pred = br.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: 0.08845326380264507
MAE: 0.09684434561297495
MSE: 0.01629700723605431


In [86]:
from sklearn.linear_model import SGDRegressor
sgd = SGDRegressor()
sgd.fit(training_data, training_target)

print('R squared: ' + str(sgd.score(test_data, test_target)))
pred = sgd.predict(test_data)
print('MAE: ' + str(metrics.mean_absolute_error(test_target, pred)))
print('MSE: ' + str(metrics.mean_squared_error(test_target, pred)))

R squared: 0.042888133821536334
MAE: 0.10017137972692909
MSE: 0.017111639359154922


In [ ]:
\begin{table}[ht]
\caption{Model Results} % title of Table
\centering % used for centering table
\begin{tabular}{c c c c} % centered columns (4 columns)
\hline\hline %inserts double horizontal lines
Case & Method\#1 & Method\#2 & Method\#3 \\ [0.5ex] % inserts table
%heading
\hline % inserts single horizontal line
1 & 50 & 837 & 970 \\ % inserting body of the table
2 & 47 & 877 & 230 \\
3 & 31 & 25 & 415 \\
4 & 35 & 144 & 2356 \\
5 & 45 & 300 & 556 \\ [1ex] % [1ex] adds vertical space
\hline %inserts single line
\end{tabular}
\label{table:nonlin} % is used to refer this table in the text
\end{table}
--------------------- e

In [64]:
from sklearn.model_selection import RandomizedSearchCV
# Setup the parameters and distributions to sample from: param_dist
param_dist_dt = {"max_depth": [1, 32],
              "max_features": list(range(1, 21)),
              "random_state": list(range(1, 5000)),
              "min_samples_split": np.linspace(0.1, 1, 10, endpoint=True),
              "min_samples_leaf": np.linspace(0.1, 0.5, 5, endpoint=True),
              "criterion": ["friedman_mse", "mse"]}
## Instantiate the RandomizedSearchCV object: tree_cv
dt_cv = RandomizedSearchCV(DecisionTreeRegressor(), \
                           param_dist_dt, cv=10, n_jobs=-1, verbose=1)
# Fit it to the data
dt_cv.fit(train_data_array, train_class_array)
# Print the tuned parameters and score
print("Tuned Decision Tree Parameters: {}".format(dt_cv.best_params_))
print("Best score is {}".format(dt_cv.best_score_))
# train mse
pred = dt_cv.predict(train_data_array)
score = metrics.mean_squared_error(train_class_array, pred)
print("Train MSE:   %0.3f" % score)
# test mse
pred = dt_cv.predict(test_data_array)
score = metrics.mean_squared_error(test_class_array, pred)
print("Test MSE:   %0.3f" % score)
# test mae
pred = dt_cv.predict(test_data_array)
score = metrics.mean_absolute_error(test_class_array, pred)
print("Test MAE:   %0.3f" % score)
# r-squared
score = metrics.r2_score(test_class_array, pred)
print("R-squared  %0.3f" % score)

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Tuned Decision Tree Parameters: {'random_state': 1558, 'min_samples_split': 0.9, 'min_samples_leaf': 0.30000000000000004, 'max_features': 2, 'max_depth': 32, 'criterion': 'mse'}
Best score is 0.06330274150685954
Train MSE:   0.014
Test MSE:   0.014
Test MAE:   0.093
R-squared  0.040


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.0s finished
